# Import Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv(r'D:\DataScienceClass\GitDemo\Rajlaxmi_kaggle\train.csv')
test_data = pd.read_csv(r'D:\DataScienceClass\GitDemo\Rajlaxmi_kaggle\test.csv')
submission_data=pd.read_csv(r'D:\DataScienceClass\GitDemo\Rajlaxmi_kaggle\sample.csv')

# EDA

# Text Preprocessing

# Model Building